# SQL Query Agent with Ollama - Exploration Notebook

**Sprint 1 Deliverable** | **DSM 1.0 Track**

This notebook builds and evaluates a natural language to SQL agent using open-source LLMs running locally via Ollama. It serves as a text-to-code generation testbed -- SQL is a constrained language ideal for systematic evaluation.

**Architecture:** LangGraph state graph with schema filtering, SQL validation (sqlglot), and error-correction retry loop. Design informed by DIN-SQL, MAC-SQL, and CHESS research.

**Goals:**
1. Verify environment (Ollama, models, database)
2. Build LangGraph agent: `schema_filter` → `generate_sql` → `validate_query` → `execute_query` → `handle_error`
3. Evaluate at least 2 models (sqlcoder:7b vs llama3.1:8b) on a curated test suite

**References:**
- Research: `docs/research/text_to_sql_state_of_art.md`
- Plan: `docs/plans/PLAN.md`
- Sprint plan: `docs/plans/sprint-1-plan.md`
